In [298]:
import functools
import numpy as np
import operator
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.sparse as sparse

import os
import sys
import inspect
import importlib
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from network import vlayers
importlib.reload(vlayers)
pass

In [452]:
def flatten(inputs, dims_to_flatten, dims_to_save=0):
    """Flatten given dimensions of tensor"""
    input_shape = inputs.shape
    rank = input_shape.rank
    batch_dims = input_shape[:rank-dims_to_flatten]
    non_batch_dims = input_shape[-dims_to_flatten:]
    
    if dims_to_save > 0:
        saved_dims = input_shape[-1:]
    
    if tf.executing_eagerly():
        # Full static shape is guaranteed to be available.
        # Performance: Using `constant_op` is much faster than passing a list.
        if dims_to_save:
            flattened_shape = tf.concat([batch_dims, [-1], saved_dims], 0)
        else:
            flattened_shape = tf.concat([batch_dims, [-1]], 0)
        return tf.reshape(inputs, flattened_shape)
    else:
        last_dim = int(functools.reduce(operator.mul, non_batch_dims))
        if dims_to_save:
            flattened_shape = tf.concat([[-1], batch_dims[1:], [last_dim], saved_dims], 0)
        else:
            flattened_shape = tf.concat([[-1], batch_dims[1:], [last_dim]], 0)
        return tf.reshape(inputs, flattened_shape)

def concat_biases(inputs, axis=-1):
    """Add bias to each input vector"""
    inputs_rank = len(inputs.shape)
    # Inputs shape can be partially known, so
    # Get inputs slice with current dimension equals one
    slice_begin = tf.zeros(inputs_rank, dtype=tf.int32)
    slice_size = tf.concat([tf.fill([inputs_rank + axis], -1), tf.constant([1]), tf.fill([-axis - 1], -1)], 0)
    inputs_slice = tf.slice(inputs, slice_begin, slice_size)
    # Create biases shaped like inputs slice
    biases = tf.ones_like(inputs_slice, dtype=inputs.dtype)
    # Concatenate inputs with biases
    x = tf.concat([inputs, biases], axis)
    
    return x
    
def get_input_shape(input_shape, paddings):
    """Get shape of input feature tensor"""
    input_shape = tf.constant(input_shape[-3:])
    if paddings is not None:
        paddings = tf.reduce_sum(paddings[-3:], axis=-1)
        input_shape += paddings
    return input_shape

def get_full_output_shape(input_shape, kernel_shape, strides, paddings, use_bias):
    """Get shape of output tensor"""
    input_shape = get_input_shape(input_shape, paddings)
    vector_dim = tf.reduce_prod(kernel_shape[:-1])
    if use_bias:
        vector_dim += 1
    
    filters_num = kernel_shape[-1]
    
    input_shape = tf.constant(input_shape[-3:])
    kernel_shape = tf.constant(kernel_shape[-4:-1])
    strides = tf.concat([tf.constant(strides), [1]], axis=0)
    
    # Convolution layer output shape formula
    output_shape = ((input_shape - kernel_shape) // strides) + 1
    # Add filters 
    output_shape *= tf.concat([1, 1, filters_num], axis=0)
    # Add vector dimension
    output_shape = tf.concat([[-1, vector_dim], output_shape], axis=0)
    return output_shape

def get_output_shape_for_single_filter(input_shape, kernel_shape, strides):
    """Get shape of output feature tensor for single filter"""
    input_shape = tf.constant(input_shape[-3:])
    kernel_shape = tf.constant(kernel_shape[-4:-1])
    strides = tf.concat([tf.constant(strides), [1]], axis=0)
    # Convolution layer output shape formula
    output_shape = ((input_shape - kernel_shape) // strides) + 1
    return output_shape

def iterate_input_gather_indices(weight_shape, input_shape, output_shape, kernel_shape, strides, use_bias, is_vector_input):
    """Iterate over indices of elements (to extract from input/activization tensor) for furhter multiplication by weights"""
    # Amounts of already taken elements in flattened input
    position_nums = dict()
    
    for j in range(weight_shape[-2]):
        # Compute position in output tensor
        chan_num = j % output_shape[-1]
        col_num = (j // output_shape[-1]) % output_shape[-2]
        row_num = (j // output_shape[-1]) // output_shape[-2]
        # Compute row in weight tensor to start with
        offset = (row_num * input_shape[-2] * strides[-2] + col_num \
                * strides[-1]) * input_shape[-1] + chan_num
        
        for i in range(tf.cast(weight_shape[-3], tf.int32) - offset):
            # Compute position in input tensor
            chan_num = i % input_shape[-1]
            col_num = (i // input_shape[-1]) % input_shape[-2]
            row_num = (i // input_shape[-1]) // input_shape[-2]
            
            if (chan_num < kernel_shape[-2] and col_num < kernel_shape[-3] and row_num < kernel_shape[-4]):
                position = (i + int(offset),)
                if is_vector_input:
                    if position not in position_nums:
                        position_nums[position] = 0
                    yield position + (position_nums[position],)
                    position_nums[position] += 1
                else:
                    yield position
        # Set bias
        if use_bias:
            if is_vector_input:
                yield (int(weight_shape[-3]-1), 0)
            else:
                yield (int(weight_shape[-3]-1),)

def get_input_gather_indices(input_shape, kernel_shape, strides, paddings, use_bias, is_vector_input):
    """Iterate over indices of elements (to extract from input/activization tensor) for furhter multiplication by weights"""
    # Padded input shape
    input_shape = get_input_shape(input_shape, paddings)
    # Output shape
    output_shape = get_output_shape_for_single_filter(input_shape, kernel_shape, strides)
    output_flat_len = np.prod(output_shape)
    
    # Compute all offsets
    row_offsets = np.arange(output_shape[-3]) * input_shape[-2].numpy() * strides[-2] * input_shape[-1].numpy()
    col_offsets = np.arange(output_shape[-2]) * strides[-1] * input_shape[-1].numpy()
    chan_offsets = np.arange(output_shape[-1])
    offsets = row_offsets[:, np.newaxis, np.newaxis] + col_offsets[:, np.newaxis] + chan_offsets
    offsets = offsets.flatten()
    
    # Find gathering indices without offset
    index_bool_vector = np.zeros(input_shape)
    index_bool_vector[:kernel_shape[-4],:kernel_shape[-3],:kernel_shape[-2]] = 1
    index_bool_vector = index_bool_vector.flatten()
    index_vector = np.nonzero(index_bool_vector)[0]
    index_matrix = index_vector[:, np.newaxis]
    index_matrix = np.tile(index_matrix, (1, output_flat_len))
    
    # Find gathering indices with offsets
    offset_index_matrix = index_matrix + offsets
    
    # Append bias
    if use_bias:
        bias_index = np.prod(input_shape)
        bias = np.full((1, offset_index_matrix.shape[-1]), bias_index, dtype=np.int32)
        offset_index_matrix = np.concatenate((offset_index_matrix, bias), axis=0)
    
    # Set vector indices for all inputs
    if is_vector_input:
        row_num = index_bool_vector.shape[0]
        col_num = output_flat_len
        row_indices = np.arange(row_num)[:,np.newaxis]
        col_indices = np.arange(col_num)
        shifts = np.mod(row_indices - offsets, row_num)
        unshifts = np.mod(row_indices + offsets, row_num)
        # Set indices
        uniqueness_indices = index_bool_vector[:, np.newaxis]
        uniqueness_indices = np.tile(uniqueness_indices, (1, col_num))
        # Shift by offsets
        uniqueness_indices = uniqueness_indices[shifts, col_indices]
        # Get unique indices
        uniqueness_indices = np.cumsum(uniqueness_indices, axis=1)
        # Unshift
        uniqueness_indices = uniqueness_indices[unshifts, col_indices]
        # Slice and convert to zero-based indexing
        uniqueness_indices = uniqueness_indices[index_vector] - 1
        # Append bias
        if use_bias:
            bias = np.full((1, uniqueness_indices.shape[-1]), 0, dtype=np.int32)
            uniqueness_indices = np.concatenate((uniqueness_indices, bias), axis=0)
        
        # Set unique vector indices
        offset_index_matrix = np.stack((offset_index_matrix, uniqueness_indices), axis=-1)
    else:
        offset_index_matrix = offset_index_matrix[...,np.newaxis]
    
    return tf.constant(offset_index_matrix, dtype=tf.int64)
        
                
def get_input_gather_indices_slow(input_shape, kernel_shape, strides, paddings, use_bias, is_vector_input):
    # Padded input shape
    input_shape = get_input_shape(input_shape, paddings)
    # Output shape
    output_shape = get_output_shape_for_single_filter(input_shape, kernel_shape, strides)
    
    # Shape of sparsed weight tensor (which actually is not used)
    input_flat_len = tf.reduce_prod(input_shape) + (1 if use_bias else 0)
    output_flat_len = tf.reduce_prod(output_shape)
    filters_len = tf.reduce_prod(kernel_shape[-1:])
    weight_shape = tf.concat([input_flat_len, output_flat_len, filters_len], axis=0)
    
    # Get indices of elements to take from input to avoid slow multiplication to sparsed tensor
    gather_indices = tf.constant(
        list(iterate_input_gather_indices(
            weight_shape, 
            input_shape, 
            output_shape, 
            kernel_shape, 
            strides, 
            use_bias, 
            is_vector_input)
        ),
        dtype=tf.int64
    )
    
    # Reshape indices to appropriate tensor
    vector_dim = tf.reduce_prod(kernel_shape[:-1])
    if use_bias:
        # Bias must be taken into account
        vector_dim += 1
    gather_shape = tf.concat([tf.reduce_prod(output_shape), vector_dim, tf.constant(2 if is_vector_input else 1)], axis=0)
    gather_indices = tf.reshape(gather_indices, gather_shape)
    gather_indices = tf.transpose(gather_indices, perm=[1,0,2])
    
    return gather_indices

def apply_convolution_kernel(x, flattened_weight, weight_tiles, gather_indices, paddings, use_bias, output_shape, is_vector_input):
    """Apply convolution kernel to input/activation"""
    # Pad
    if paddings is not None:
        x = tf.pad(x, paddings)
    # Flatten
    x_flat = flatten(x, tf.constant(3), 1 if is_vector_input else 0)
    if use_bias:
        x_flat = concat_biases(x_flat, axis=-1)
    # Rearrange
    x = tf.transpose(x_flat, perm=[1,0])
    x = tf.gather_nd(x, gather_indices)
    x = tf.transpose(x, perm=[2,0,1])
    x = tf.expand_dims(x, -1)
    # Fill weight tensor
    weight = tf.tile(flattened_weight, weight_tiles)
    # Multiply 
    y = x * weight
    # Reshape
    return tf.reshape(y, output_shape)

def apply_padding_kernel(x, gather_indices, paddings, use_bias, output_shape):
    """Apply padding (pseudo)kernel to input/activation"""
    # Pad
    if paddings is not None:
        x = tf.pad(x, paddings)
    # Flatten
    x_flat = flatten(x, tf.constant(3))
    if use_bias:
        x_flat = concat_biases(x_flat, axis=-1)
    # Rearrange
    x = tf.transpose(x_flat, perm=[1,0])
    x = tf.gather_nd(x, gather_indices)
    x = tf.transpose(x, perm=[2,0,1])
    x = tf.expand_dims(x, -1)
    # Reshape
    return tf.reshape(x, output_shape)

def do_convolution(x, activation_fun):
    """Do simple convolution over input"""
    # Sum
    s = tf.reduce_sum(x, -4)
    # Compute activation
    a = activation_fun(s)
    
    return a

def do_convolution_with_inner_net(x, inner_input, inner_hiddens, inner_output):
    """Do convolution over input, using inner fractal network"""
    # Set vector input as last dimension
    x = tf.transpose(x, perm=[0,2,3,4,1])
    # Process data as inner network
    y = self.inner_input(x)
    for hidden in self.inner_hiddens:
        y = hidden(y)
    a = self.inner_output(y)
    
    return a

def do_pooling(x, pooling_fun):
    """Do pooling over input"""
    s = pooling_fun(x, -4)
    
    return s

In [454]:
class VInputConv(layers.Layer):
    """Input vector layer for convolutional networks"""
    def __init__(self, filter_shape, num_filters=1, kernel_type="convolution", strides=(1,1), padding_type="valid", weight_initializer="random_normal"):
        super().__init__()
        self.filter_shape = filter_shape
        self.num_filters = num_filters
        self.kernel_type = kernel_type
        self.strides = strides
        self.padding_type = padding_type
        self.weight_initializer = weight_initializer
        self.is_vector_input = False
    
    def build(self, input_shape):
        input_rank = input_shape.rank
        
        # Set kernel shape
        if self.kernel_type == "convolution":
            kernel_shape = tf.concat([self.filter_shape, input_shape[-1:], [self.num_filters]], axis=0)
            use_bias = True
            bias_shape = kernel_shape[-1:]
        else: # elif kernel_type == "pooling"
            kernel_shape = tf.concat([self.filter_shape, [1, 1]], axis=0)
            use_bias = False
        
        # Init padding
        if self.padding_type == "same":
            paddings = tf.constant(kernel_shape[-4:-2]) - 1
            paddings_before = paddings // 2
            paddings_after = paddings - paddings_before
            paddings_before = tf.concat([tf.zeros([input_rank - 3], dtype=tf.int32), paddings_before, [0]], axis=0)
            paddings_after = tf.concat([tf.zeros([input_rank - 3], dtype=tf.int32), paddings_after, [0]], axis=0)
            paddings = tf.stack([paddings_before, paddings_after], axis=1)
        elif self.padding_type == "full":
            paddings = tf.constant(kernel_shape[-4:-2]) - 1
            paddings = tf.concat([tf.zeros([input_rank - 3], dtype=tf.int32), paddings, [0]], axis=0)
            paddings = tf.stack([paddings, paddings], axis=1)
        else: # elif self.padding_type == "valid"
            paddings = None
        
        # Set output shape
        self.output_shape = get_full_output_shape(input_shape, kernel_shape, self.strides, paddings, use_bias)
        
        # Get indices to gather elements input/activation tensor into appropriate shape (rearrange)
#         gather_indices = get_input_gather_indices_slow(
#             input_shape, 
#             kernel_shape, 
#             self.strides, 
#             paddings, 
#             use_bias, 
#             self.is_vector_input
#         )
        gather_indices = get_input_gather_indices(
            input_shape, 
            kernel_shape, 
            self.strides, 
            paddings, 
            use_bias, 
            self.is_vector_input
        )
        
        if self.kernel_type == "convolution":
            # Init weights and biases (already flattened)
            flattened_weight_shape = tf.concat(
                [
                    tf.reduce_prod(kernel_shape[:-1]) + 1 if use_bias else 0, 
                    1, 
                    kernel_shape[-1]
                ], 
                axis=0
            )
            self.flattened_weight = self.add_weight(
                shape=flattened_weight_shape,
                initializer=self.weight_initializer
            )
#             if self.kernel is None:
#                 self.kernel = self.add_weight(
#                     shape=kernel_shape,
#                     initializer=self.weight_initializer
#                 )
#                 if use_bias:
#                     self.bias = self.add_weight(
#                         shape=bias_shape,
#                         initializer=self.weight_initializer
#                     )

#             # Get flattened kernel
#             self.flattened_weight = tf.reshape(self.kernel, tf.concat([tf.reduce_prod(self.kernel.shape[:-1]), 1, self.kernel.shape[-1]], axis=0))
#             if use_bias:
#                 # Get bias values
#                 bias = tf.reshape(self.bias, tf.concat([1, 1, self.bias.shape[-1]], axis=0))
#                 self.flattened_weight = tf.concat([self.flattened_weight, bias], axis=0)
            
            # Dimensions to tile weights
            weight_tiles = tf.concat([[1], gather_indices.shape[-2:-1], [1]], axis=0)
            
            self.weight_multiply_fun = lambda x: apply_convolution_kernel(
                x, self.flattened_weight, weight_tiles, gather_indices, paddings, use_bias, self.output_shape
            )
        else: # elif kernel_type == "pooling"
            self.weight_multiply_fun = lambda x: apply_padding_kernel(
                x, gather_indices, paddings, use_bias, self.output_shape
            )
    
    def call(self, inputs):
        return self.weight_multiply_fun(inputs)

        
class VConv(VInputConv):
    """Hidden vector convolution layer"""
    def __init__(self, filter_shape, num_filters=1, kernel_type="convolution", strides=(1,1), padding_type="valid", weight_initializer="random_normal", layer_type="convolution", activation="relu", pooling="max"):
        super().__init__(filter_shape, num_filters, kernel_type, strides, padding_type, weight_initializer)
        
        self.layer_type = layer_type
        self.activation = activation
        self.pooling = pooling
        self.activation_fun = tf.keras.activations.deserialize(activation)
        if pooling == "max":
            self.pooling_fun = tf.math.reduce_max
        elif pooling == "mean":
            self.pooling_fun = tf.math.reduce_mean
    
    def build(self, input_shape):
        super().build(input_shape)
        
        if self.layer_type == "convolution":
            self.op_fun = lambda x: do_convolution(x, self.activation_fun, self.is_vector_input)
        else: # elif self.layer_type == "pooling"
            self.op_fun = lambda x: do_pooling(x, self.pooling_fun)
    
    def call(self, inputs):
        x = self.op_fun(inputs)
        return self.weight_multiply_fun(x)
    

class VConvFractal(VConv):
    """Hidden vector convolution layer with inner networks"""
    def __init__(self, filter_shape, num_filters=1, kernel_type="convolution", strides=(1,1), padding_type="valid", weight_initializer="random_normal", layer_type="convolution", activation="relu", pooling="max", depth=1, shared_inner_nets=False, hidden_layer_units=(2,)):
        super().__init__(
            filter_shape, 
            num_filters, 
            kernel_type, 
            strides, 
            padding_type, 
            weight_initializer, 
            layer_type, 
            activation, 
            pooling
        )
        self.is_vector_input = True
        self.depth = depth
        self.shared_inner_nets = shared_inner_nets
        self.hidden_layer_units = hidden_layer_units
    
    def build(self, input_shape):
        super().build(input_shape)
        
        # Initialize inner network
        if layer_type == "convolution" and self.depth > 0:
            input_item_rank = 2 if self.shared_inner_nets else 1
            output_units = self.output_shape[-4]
            
            self.inner_input = VInput(self.hidden_layer_units[0], 
                                      input_item_rank=input_item_rank,
                                      add_bias=False, 
                                      weight_initializer=self.weight_initializer,
                                      weight_type="unique")
            self.inner_hiddens = [VFractal(u, 
                                           input_item_rank = input_item_rank,
                                           depth=self.depth - 1,
                                           hidden_layer_units=self.hidden_layer_units,
                                           activation=self.activation,
                                           weight_initializer=self.weight_initializer,
                                           weight_type="unique") 
                                  for u 
                                  in self.hidden_layer_units[1:] + (output_units,)]
            self.inner_output = VOutput(activation=self.activation)
            
            self.op_fun = lambda x: do_convolution_with_inner_net(x, self.inner_input, self.inner_hiddens, self.inner_output)
    
    def call(self, inputs):
        return super().call(inputs)
        
    
class VOutputConv(layers.Layer):
    """Output vector layer for convolutional networks"""
    def __init__(self, layer_type="convolution", activation="relu", pooling="max"):
        super().__init__()
        self.layer_type = layer_type
        self.activation = activation
        self.pooling = pooling
        self.activation_fun = tf.keras.activations.deserialize(activation)
        if pooling == "max":
            self.pooling_fun = tf.math.reduce_max
        elif pooling == "mean":
            self.pooling_fun = tf.math.reduce_mean
    
    def build(self, input_shape):
        if self.layer_type == "convolution":
            self.op_fun = lambda x: do_convolution(x, self.activation_fun)
        else: # elif self.layer_type == "pooling"
            self.op_fun = lambda x: do_pooling(x, self.pooling_fun)
    
    def call(self, inputs):
        return self.op_fun(inputs)

In [447]:
import timeit
batch_size = 64
x_dim = 32
x_shape=3
x = tf.reshape(tf.range([batch_size*x_dim*x_dim*x_shape], dtype=tf.float32), shape=(batch_size,x_dim,x_dim,x_shape))
strides = (1,1)
padding = "same"
num_filters=12
filter_dim=7

tries = 1000

In [448]:
platform_layer1 = layers.Conv2D(num_filters, filter_dim, activation='relu', strides=strides, padding=padding)
platform_layer2 = tf.keras.layers.ReLU()
platform_layer3 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(1, 1))
x1 = platform_layer1(x)
x1 = platform_layer2(x1)
x1 = platform_layer3(x1)
print(x1.shape)

def platform_net():
    x1 = platform_layer1(x)
    x1 = platform_layer2(x1)
    x1 = platform_layer3(x1)

(64, 31, 31, 12)


In [118]:
print(platform_layer1.get_weights()[0].shape)

(3, 3, 1, 1)


In [430]:
vlayer1 = VInputConv((filter_dim, filter_dim), num_filters, "convolution", strides, padding_type=padding)
# vlayer1.kernel = tf.reshape(tf.range(filter_dim*filter_dim*x_shape*num_filters, dtype=tf.float32)*0.01, shape=(filter_dim,filter_dim,x_shape,num_filters))
# print(platform_layer1.get_weights()[0].shape)
# print(tf.reshape(tf.range(filter_dim*filter_dim*x_shape*num_filters, dtype=tf.float32)*0.01, shape=(filter_dim,filter_dim,x_shape,num_filters)).shape)
vlayer1.kernel = platform_layer1.get_weights()[0]
# vlayer1.bias = tf.ones([num_filters], dtype=tf.float32)
vlayer1.bias = platform_layer1.get_weights()[1]
vlayer2 = VConv((2,2), kernel_type="pooling", strides=(1,1), padding_type="valid", layer_type="convolution", activation="relu")
vlayer3  = VOutputConv(layer_type="pooling", pooling="max")
x1 = vlayer1(x)
x1 = vlayer2(x1)
vlayer3(x1)

<tf.Tensor: shape=(1, 2, 2, 2), dtype=float32, numpy=
array([[[[3.218261 , 4.119361 ],
         [0.       , 4.119361 ]],

        [[5.401186 , 3.3237386],
         [0.       , 1.6719575]]]], dtype=float32)>

In [449]:
vlayer1 = VInputConv((filter_dim, filter_dim), num_filters, "convolution", strides, padding_type=padding)
vlayer2 = VConv((2,2), kernel_type="pooling", strides=(1,1), padding_type="valid", layer_type="convolution", activation="relu")
vlayer3  = VOutputConv(layer_type="pooling", pooling="max")
# v_time = timeit.timeit(lambda: print(vlayer3(vlayer2(vlayer1(x))).shape), number=1)
x1 = vlayer1(x)
x1 = vlayer2(x1)
x1 = vlayer3(x1)
print(x1.shape)

def vnet():
    x1 = vlayer1(x)
    x1 = vlayer2(x1)
    x1 = vlayer3(x1)

(64, 31, 31, 12)


In [297]:
v_time

794.1633896999992

In [450]:
v_time = timeit.timeit(vnet, number=tries)
platform_time  = timeit.timeit(platform_net, number=tries)

print(v_time / platform_time)

8.315365029161324
